#Ejemplo de procesado y visualización de datos estadísticos de Cantabria usando la web del ICANE

En el siguiente cuaderno se muestran tres ejemplos sencillos de manipulación y representación de datos obtenidos a través de las interfaces web del [ICANE](http://www.icane.es/metadata). Para ello se ha utilizado [Pandas](http://pandas.pydata.org/), la biblioteca de análisis de datos por excelencia en Python; [pyicane](https://github.com/predicador37/pyicane), una biblioteca desarrollada por personal del ICANE para facilitar la extracción automática de datos a partir del formato [JSON](http://es.wikipedia.org/wiki/JSON) y [Vincent](https://github.com/wrobstory/vincent), una librería de visualización que abstrae la complejidad de [Vega](http://trifacta.github.io/vega/) (JSON), a su vez abstracción de [D3.js](http://d3js.org/) en [Javascript](http://es.wikipedia.org/wiki/JavaScript).

##Ejemplo de obtención de datos

En primer lugar, se mostrará una forma sencilla de extraer datos de forma automatizada usando la API de datos del ICANE.

Para ello, se tomará como ejemplo la serie de [Revisiones Anuales del Padrón Municipal de Habitantes](http://www.icane.es/data/municipal-register-annual-review-municipality#timeseries).

In [2]:
from pyicane import pyicane
from collections import OrderedDict

#Obtención de datos de Revisión Anual del PMH desde la web del ICANE
rapmh = pyicane.TimeSeries.get('municipal-register-annual-review-municipality')

#Conversión a dataframe
data = rapmh.data_as_dataframe()

#Filtrado de variables y año
varint = data[(data['Variables'] == 
                  'Variación interanual') & (data.index == '2014')]

#Filtrado del total Cantabria
varint = varint.ix[:-1]

#Obtención de código INE y nombre de municipio en columnas separadas
cod_ine = [item.split(' - ')[0].lstrip() for item in varint['Municipios']]
nombres = [item.split(' - ')[1].lstrip() for item in varint['Municipios']]
varint['cod_ine'] = cod_ine
varint['nombres'] = nombres

#Conversión automática de tipos de datos
varint = varint.convert_objects(convert_numeric=True)
varint.head()

,Variables,Municipios,Valor,cod_ine,nombres
Año,,,,,
2014,Variación interanual,39001 - Alfoz de Lloredo,-1.23,39001,Alfoz de Lloredo
2014,Variación interanual,39002 - Ampuero,-0.63,39002,Ampuero
2014,Variación interanual,39003 - Anievas,-1.80,39003,Anievas
2014,Variación interanual,39004 - Arenas de Iguña,-1.99,39004,Arenas de Iguña
2014,Variación interanual,39005 - Argoños,-0.35,39005,Argoños


##Mapa de coropletas
A continuación, se mostrará un ejemplo de representación de datos en mapas en forma de coropletas.

Para este ejemplo, se ha utilizado un mapa en formato [GeoJSON](http://geojson.org/) realizado con datos del Gobierno de Cantabria-IGN-CNIG, límites inscritos en el Registro Central de Cartografía a fecha 13 de Diciembre de 2012 y proporcionado por la [Consejería de Medio Ambiente, Ordenación del Territorio y Urbanismo](http://cartografia.cantabria.es/Descargas).

Con objeto de obtener una representación más manejable, previamente se convirtió el mapa a formato [TopoJSON](https://github.com/mbostock/topojson), utilizado posteriormente para dibujarlo. Para servirlo, se ha subido este archivo a la [nube S3](http://d1jbhcb2qvxl2c.cloudfront.net/cantabria.topo.json) de Amazon.

Concretamente lo que se pretende representar es la tasa de variación anual de población según la serie anterior.

In [3]:
import vincent
from vincent.values import ValueRef
from IPython.display import display
import requests

vincent.core.initialize_notebook()

#Obtención de los datos del mapa municipal en topojson
cantabria_topo = r'http://d1jbhcb2qvxl2c.cloudfront.net/cantabria.topo.json'
geo_data = [{'name': 'cantabria',
             'url': cantabria_topo,
             'feature': 'cantabria'}]

#Creación del gráfico y asociación de los datos al mapa
vis = vincent.Map(geo_data=geo_data, data=varint, scale=20000, 
                  projection = 'mercator', center = [-4.00, 43.13],
                  data_bind='Valor', data_key='cod_ine',
                  map_key={'cantabria': 'properties.COD_INE'})
vis.marks[0].properties.enter.stroke_opacity = ValueRef(value=0.5)
vis.legend(title="Revisión anual PMH: variación interanual de población")
display(vis)

##Operaciones con datos

En las siguientes secciones se observará cómo trabajar con datos ya extraídos automáticamente. En concreto, se realizarán operaciones de filtrado, limpieza, pivotaje y manipulación de datos en general.

Para estos ejemplos se utilizará la serie de [Precios corrientes](http://www.icane.es/data/quarterly-accounting-cantabria-base-2008-current-prices#content) de la Contabilidad trimestral de Cantabria, Base 2008 y CNAE 2009.

In [4]:
import datetime as dt
import calendar
import time

#Obtención de datos de CTC desde la web del ICANE
contrim_raw = pyicane.TimeSeries.get('quarterly-accounting-cantabria-base-2008-current-prices')
contrim_df = contrim_raw.data_as_dataframe()

#Filtrado de datos para conseguir:
# Tasa de variación interanual
# Ajustada de estacionalidad y efecto calendario
# PIB total
tvarint = contrim_df[(contrim_df['Variables'] == 
                  'Tasa de variación interanual') & (contrim_df['Tipo de ajuste'] == 
                  'Ajustada de estacionalidad y efecto calendario')& (contrim_df['Sector'] == 
                  '1 PIB total')]

#Filtrado de filas con valor desconocido
tvarint = tvarint[tvarint['Valor']!='..'] 
tvarint.head()

,Sector,Tipo de ajuste,Variables,Valor
Trimestre,,,,
2001 - 1,1 PIB total,Ajustada de estacionalidad y efecto calendario,Tasa de variación interanual,5.7
2001 - 2,1 PIB total,Ajustada de estacionalidad y efecto calendario,Tasa de variación interanual,8.7
2001 - 3,1 PIB total,Ajustada de estacionalidad y efecto calendario,Tasa de variación interanual,9.4
2001 - 4,1 PIB total,Ajustada de estacionalidad y efecto calendario,Tasa de variación interanual,9.3
2002 - 1,1 PIB total,Ajustada de estacionalidad y efecto calendario,Tasa de variación interanual,8.8


Una vez extraídos y filtrados los datos, será necesario crear un índice temporal a partir de los datos de año y trimestre:

In [5]:
#Extracción del año y el trimestre y generación de fechas
anyo, trimestre = zip(*(s.split(" - ") for s in list(tvarint.index)))
fechas = [dt.datetime(int(y) , int(q)*3, int(calendar.monthrange(int(y), int(q)*3)[1])) for q,y in zip(trimestre,anyo)]

#Adición de columna de fechas a la serie
tvarint['fechas'] = fechas

#Pivotaje para obtener una columna por categoría
tvarint = tvarint.pivot(index='fechas',columns='Sector', values='Valor')
tvarint.head()

Sector,1 PIB total
fechas,
2001-03-31,5.7
2001-06-30,8.7
2001-09-30,9.4
2001-12-31,9.3
2002-03-31,8.8


##Gráfico de línea

Es posible utilizar los datos obtenidos para representar gráficamente la serie temporal a través de un gráfico de línea y utilizando un etiquetado sencillo:

In [6]:
#Generación del gráfico de serie temporal
lineas = vincent.Line(tvarint)
lineas.scales[0].type = 'time'
lineas.axis_titles(x='Tiempo', y='Valor')
lineas.legend(title='PIB total')
display(lineas)

##Gráfico multilínea

Finalmente, se generará un nuevo conjunto de datos seleccionando los sectores de Industria y energía, construcción y servicios. A partir de dicho conjunto, realizando operaciones similares a las detalladas anteriormente, es posible obtener una representación multilínea de la serie temporal categorizada según dichos sectores:

In [7]:
import datetime as dt
import calendar
import time

#Filtrado de datos para conseguir:
# Tasa de variación interanual
# Ajustada de estacionalidad y efecto calendario
# Sectores Industria, construcción y servicios
tvarint_multi = contrim_df[(contrim_df['Variables'] == 
                  'Tasa de variación interanual') & (contrim_df['Tipo de ajuste'] == 
                  'Ajustada de estacionalidad y efecto calendario')& ((contrim_df['Sector'] == 
                  '3 Industria y energía') | (contrim_df['Sector'] == 
                  '4 Construcción') | (contrim_df['Sector'] == 
                  '5 Servicios'))]

#Filtrado de filas con valor desconocido
tvarint_multi = tvarint_multi[tvarint_multi['Valor']!='..'] 
tvarint_multi.head()

,Sector,Tipo de ajuste,Variables,Valor
Trimestre,,,,
2001 - 1,3 Industria y energía,Ajustada de estacionalidad y efecto calendario,Tasa de variación interanual,2.7
2001 - 2,3 Industria y energía,Ajustada de estacionalidad y efecto calendario,Tasa de variación interanual,8
2001 - 3,3 Industria y energía,Ajustada de estacionalidad y efecto calendario,Tasa de variación interanual,8.8
2001 - 4,3 Industria y energía,Ajustada de estacionalidad y efecto calendario,Tasa de variación interanual,7.6
2002 - 1,3 Industria y energía,Ajustada de estacionalidad y efecto calendario,Tasa de variación interanual,6.9


In [8]:
#Extracción del año y el trimestre y generación de fechas
anyo, trimestre = zip(*(s.split(" - ") for s in list(tvarint_multi.index)))
fechas = [dt.datetime(int(y) , int(q)*3, int(calendar.monthrange(int(y), int(q)*3)[1])) for q,y in zip(trimestre,anyo)]

#Adición de columna de fechas a la serie
tvarint_multi['fechas'] = fechas

#Pivotaje para obtener una columna por categoría
tvarint_multi = tvarint_multi.pivot(index='fechas',columns='Sector', values='Valor')
tvarint.head()

Sector,1 PIB total
fechas,
2001-03-31,5.7
2001-06-30,8.7
2001-09-30,9.4
2001-12-31,9.3
2002-03-31,8.8


In [9]:
#Generación de gráfico multi-línea
multi = vincent.Line(tvarint_multi)
multi.scales[0].type = 'time'
multi.axis_titles(x='Tiempo', y='Valor')
multi.legend(title='Tasa de variación interanual')
display(multi)

Con estos ejemplos se ha pretendido ofrecer un marco introductorio para la obtención, tratamiento y representación de datos estadísticos de Cantabria utilizando el lenguaje de programación Python conjuntamente con sus bibliotecas más comunes, así como una bilioteca propia para la extracción automática de datos desde la web del ICANE.